In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import model_selection,naive_bayes,svm
import plotly
import plotly.graph_objs as go
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
import seaborn as sns
import shap
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial.distance import euclidean, cosine
import re
from utils import *
from sklearn.preprocessing import LabelEncoder
from math import log
from sklearn.svm import SVC
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import roc_auc_score
import shap
from imblearn.metrics import geometric_mean_score
from scipy.spatial import distance

Using TensorFlow backend.


In [2]:
#preprocess the text
#Make to function, preprocess, and to calculate cosine similarity
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    non_word='[^a-zA-Z]'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text=re.sub(non_word,' ',parsed_text)
    return parsed_text

In [3]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
     intersection = set(vec1.keys()) & set(vec2.keys())
     numerator = sum([vec1[x] * vec2[x] for x in intersection])

     sum1 = sum([vec1[x]**2 for x in vec1.keys()])
     sum2 = sum([vec2[x]**2 for x in vec2.keys()])
     denominator = math.sqrt(sum1) * math.sqrt(sum2)

     if not denominator:
        return 0.0
     else:
        return float(numerator) / denominator

def text_to_vector(text):
     words = WORD.findall(text)
     return Counter(words)

In [4]:
#parameter setting
C = 1
max_iter = 1000
Degree=3
Gamma='auto'

In [5]:
train=pd.read_csv("dataset/Founta_edit/train.csv")
test=pd.read_csv("dataset/Founta_edit/test.csv")
unlabel=pd.read_csv("dataset/Founta_edit/unlabel.csv")
gold_standard=pd.read_csv("dataset/Founta_edit/gold_standard.csv")

In [6]:
train.dropna(inplace=True)
test.dropna(inplace=True)
unlabel.dropna(inplace=True)

In [7]:
#mulai dari sini
print(len(train))
print(len(test))

2369
2371


In [ ]:
#training a SVM classifier and predict on unlabel pool
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text'])
x_train = Tfidf_vect.transform(train['text']).toarray()
x_test = Tfidf_vect.transform(test['text']).toarray()
x_pool=Tfidf_vect.transform(unlabel['text']).toarray()
y_train=train['label']
print("model prediction and predict on unlabel")
model = SVC( max_iter=max_iter, C=C, kernel='linear',probability=True,gamma=Gamma,degree=Degree)
model.fit(x_train, y_train)
model.score(x_train,y_train)
probs=model.predict_proba(x_pool)
entropy=list()
for p in probs:
    ent=0
    ent1=-p[0] * log(p[0],2)
    ent2=-p[1] * log(p[1],2)
    ent=ent1+ent2
    entropy.append(ent)
df=pd.DataFrame({'index':unlabel['index'],"text":unlabel['text'],"confidence":entropy})
df=df.sort_values(by="confidence",ascending=False)
unlabel_length=len(unlabel)*0.1
low_conf=df[0:round(unlabel_length)]

In [ ]:
#calculate shapley value on low conf data
x_pool=Tfidf_vect.transform(low_conf['text']).toarray()
print("calculate Shapley value ")
explainer = shap.LinearExplainer(model, x_train,feature_dependence="independent")
shap_values_train = explainer.shap_values(x_train)
shap_values_pool = explainer.shap_values(x_pool)
n_clusters=20

print("clusterint ")
kmeans = KMeans(n_clusters= n_clusters, n_jobs=-1, max_iter=600)
kmeans.fit(shap_values_pool)

#calculate the center
similarity_to_center = list()
similarity=0
for i, instance in enumerate(shap_values_pool):
    cluster_label = kmeans.labels_[i] # cluster of this instance
    centroid = kmeans.cluster_centers_[cluster_label] # cluster center of the cluster of that instance
    #similarity = 1-cosine(instance, centroid) # 1- cosine distance gives similarity
    similarity = (1)-(distance.cosine([instance], [centroid]))
    similarity_to_center.append(similarity)

In [ ]:
#inver=Tfidf_vect.inverse_transform(shap_values_pool)
#sentence=[' '.join(item) for item in inver]
#sentence

In [ ]:

label=pd.DataFrame(kmeans.labels_)
cluster_labels=pd.DataFrame({"index":low_conf['index'],"text":low_conf['text'],'cluster':kmeans.labels_,
                            'similarity':similarity_to_center,'uncertainty':low_conf['confidence']})
cluster_dict=dict()
n_cluster=20
#creating the dataframe for each cluster
for item in range(0,n_cluster):
    cluster_dict['cluster_{0}'.format(item)]=cluster_labels[cluster_labels['cluster']==item] 
#Create the dataframe for each cluster
for i in range(0,n_cluster):
    globals()['cluster_{}'.format(i)] = pd.DataFrame(cluster_dict['cluster_{}'.format(i)])
#calculate the center of the cluster
#get the center of the cluster
center=kmeans.cluster_centers_
#print(center.shape)
#create the data frame of the cluster center
centerpd=pd.DataFrame(center)

In [ ]:
#sort the value on each cluster based on similarity
for i in range(0,n_cluster):
    globals()['cluster_{}'.format(i)]=globals()['cluster_{}'.format(i)].sort_values(by='uncertainty',ascending=False)

In [ ]:
for i in range(0,n_cluster):
    print(len(globals()['cluster_{}'.format(i)]))

In [ ]:
print("cluster 5")
print("number of samples = {}".format(len(cluster_6)))
print("max_sim = {}".format(cluster_6['similarity'].max()))
print("min_sim = {}".format(cluster_6['similarity'].min()))
print("sim_range={}".format(cluster_6['similarity'].max()-cluster_6['similarity'].min()))

print("===================================================")
print("max_unc = {}".format(cluster_6['uncertainty'].max()))
print("min_unc = {}".format(cluster_6['uncertainty'].min()))
print("unc_range={}".format(cluster_6['uncertainty'].max()-cluster_6['uncertainty'].min()))



In [ ]:
#cluster_0

In [ ]:
#cluster_11.to_csv("bad_cluster2.csv",index=False)

In [ ]:
#saving the uncertainty of data points in csv file
#for i in range(0,n_cluster):
#    name_file="uncertainty/8/cluster_{0}.csv".format(i)
#    globals()['cluster_{}'.format(i)].to_csv(name_file,index=False)

In [ ]:
#exclude the bad cluster
cluster_exclude=list()
for i in range(0,20):
    if len(globals()['cluster_{}'.format(i)])>=50:
        cluster_exclude.append(i)
        del globals()['cluster_{}'.format(i)]

In [ ]:
#label propagation
#read the ground truth
search_idx=list()
gold_standard=pd.read_csv("result/gold_standard.csv")
for i in range(0,20):
    if i in cluster_exclude:
        continue
    else:
        #search for the label in the ground truth
        search_index=globals()['cluster_{}'.format(i)]['index'].head(1).values[0]
        labeled_selection=gold_standard.loc[gold_standard['index']==search_index]
        labels=labeled_selection['label'].values[0]
        globals()['cluster_{}'.format(i)]['label']=labels

In [ ]:
#get the data and balance it before put back in the training dataset
print("Balancing ")
conf_data=pd.DataFrame()
for i in range(0,20):
    if i in cluster_exclude:
        continue
    else:
        conf_data=conf_data.append(globals()['cluster_{}'.format(i)])
print(len(conf_data[conf_data['label']==0]))
print(len(conf_data[conf_data['label']==1]))
majority=conf_data[conf_data['label']==0]
n_minority_class=conf_data[conf_data['label']==1]
if len(n_minority_class) > len(majority):
    cluster_size=len(majority)
elif len(n_minority_class)==0:
    cluster_size=1
else:
    cluster_size=len(n_minority_class)

In [ ]:
#get the majority and cluster it based on number of minority class
embeddings = Tfidf_vect.transform(majority['text'])
#initialize the kmeans
kmeans = KMeans(n_clusters=cluster_size, random_state=0,max_iter=600,n_init=10)
kmeans.fit(embeddings)
labels = (kmeans.labels_)
#create the cluster for each low conf data
cluster_labels = pd.DataFrame({'index':majority['index'],'cluster':labels,'text':majority['text'],
                               'label':majority['label']})
cluster_dict=dict()
#creating the dataframe for each cluster
for item in range(0,cluster_size):
    cluster_dict['cluster_{0}'.format(item)]=cluster_labels[cluster_labels['cluster']==item] 
for i in range(0,cluster_size):
        globals()['cluster_{}'.format(i)] = pd.DataFrame(cluster_dict['cluster_{}'.format(i)])
center=kmeans.cluster_centers_
#print(center.shape)
#create the data frame of the cluster center
centerpd=pd.DataFrame(center)
#inverse transform the center to text
inver=Tfidf_vect.inverse_transform(centerpd)
#get the inverse word and create as a sentence
sentence=[' '.join(item) for item in inver]
#Calculate the data that close to the center for each cluster
for i in range(0,cluster_size):
    sent=list()
    cossim=list()
    for item in globals()['cluster_{}'.format(i)]['text']:
        sent.append(item)
    for k in sent:
        vector1=text_to_vector(k)
        vector2=text_to_vector(sentence[i])
        cosine=get_cosine(vector1,vector2)
        cossim.append(cosine)
    globals()['cluster_{}'.format(i)]['similarity']=cossim
    globals()['cluster_{}'.format(i)]=globals()['cluster_{}'.format(i)].sort_values(by='similarity',ascending=False)
center_data=pd.DataFrame()
for i in range(0,cluster_size):
    center_data=center_data.append(globals()['cluster_{}'.format(i)].head(1))

In [ ]:
#adding to training data
train_1=n_minority_class
train_0=center_data
new_train=pd.DataFrame()
#adding the balanced set for training
frames=[train_1,train_0] 

new_train=pd.concat(frames)
new_train.drop(['cluster','similarity'],axis=1,inplace=True)
training_data=pd.DataFrame({'index':new_train['index'],
                           'label':new_train['label'],'text':new_train['text']})


In [ ]:
print("minorty class {}".format(len(new_train[new_train['label']==1])))
print("majority class {}".format(len(new_train[new_train['label']==0])))

In [ ]:
#add to training data and remove from unlabel pool
train=train.append(training_data)
train.to_csv("training/training.csv",index=False)

test.to_csv("training/test.csv",index=False)

unlabel=unlabel[~unlabel['index'].isin(train['index'])]

In [ ]:
print(len(train))
print(len(test))

In [ ]:
#evaluating performance
train1=pd.read_csv("training/training.csv")
test1=pd.read_csv("training/test.csv")

train1.dropna(inplace=True)
test1.dropna(inplace=True)


X_train=train1['text']
y_train=train1['label']
X_test=test1['text']
y_test=test1['label']

Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(train['text'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)
SVM = svm.SVC(C=1, kernel='linear', gamma='auto',probability=True)
SVM.fit(Train_X_Tfidf,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
score=geometric_mean_score(y_test, predictions_SVM)
#gmean.append(score)

In [ ]:
print(score)